## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-21 05:00:46.218130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 05:00:47.082453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [46]:
task_name = "axion2"

## Loading data

See `cnn/data.py`.

In [47]:
with open(f"{data_dir}/processed/cnn/{task_name}_X_train.pkl", "rb") as fin:
    X_train = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_Y_train.pkl", "rb") as fin:
    Y_train = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_X_test.pkl", "rb") as fin:
    X_test = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_Y_test.pkl", "rb") as fin:
    Y_test = pickle.load(fin)

In [48]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (210000, 4, 16, 1) (210000, 4, 128, 1)
  labels: (210000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-21 05:00:50.327136: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-01-21 05:00:50.328471: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9804 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2024-01-21 05:00:50.329352: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9804 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3d:00.0, compute capability: 7.5
2024-01-21 05:00:50.330202: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9804 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
visible3 = Input(shape=(16, 16, 1))
conv31 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible3)
pool31 = MaxPooling2D(pool_size=(2, 2))(conv31)
conv32 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool31)
pool32 = MaxPooling2D(pool_size=(2, 2))(conv32)
flat3 = Flatten()(pool32)
print ('cov31.shape', conv31.shape)
print ('poo31.shape', pool31.shape)
print ('cov32.shape', conv32.shape)
print ('poo32.shape', pool32.shape)

cov31.shape (None, 16, 16, 32)
poo31.shape (None, 8, 8, 32)
cov32.shape (None, 8, 8, 16)
poo32.shape (None, 4, 4, 16)


In [11]:
visible4 = Input(shape=(16, 8, 1))
conv41 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible4)
pool41 = MaxPooling2D(pool_size=(2, 2))(conv41)
conv42 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool41)
pool42 = MaxPooling2D(pool_size=(2, 2))(conv42)
flat4 = Flatten()(pool32)
print ('cov41.shape', conv41.shape)
print ('poo41.shape', pool41.shape)
print ('cov42.shape', conv42.shape)
print ('poo42.shape', pool42.shape)

cov41.shape (None, 16, 8, 32)
poo41.shape (None, 8, 4, 32)
cov42.shape (None, 8, 4, 16)
poo42.shape (None, 4, 2, 16)


In [12]:
merge = concatenate([flat1, flat2, flat3, flat4])

In [49]:
# interpretation model
hidden1 = Dense(20, activation='relu')(merge)
hidden2 = Dense(20, activation='relu')(hidden1)
output = Dense(3)(hidden2)
cnn = Model(inputs=[visible1, visible2, visible3, visible4], outputs=output)

In [50]:
# summarize layers
# print(cnn.summary())

In [51]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 16, 16, 1) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 16, 8, 1) dtype=float32 (created by layer 'input_4')>]

## Train the model

In [52]:
def train_iteration(lr, epochs):
    print(f"=== Training with lr={lr} for {epochs} epochs ===")
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=128,
        validation_data=(X_test, Y_test),
    )

In [55]:
train_iteration(lr=1e-2, epochs=3)

=== Training with lr=0.01 for 3 epochs ===
Epoch 1/3
1641/1641 [==============================] - 12s 6ms/step - loss: 0.5607 - accuracy: 0.7599 - val_loss: 0.8752 - val_accuracy: 0.5164
Epoch 2/3
1641/1641 [==============================] - 9s 6ms/step - loss: 0.7410 - accuracy: 0.6152 - val_loss: 0.7866 - val_accuracy: 0.6049
Epoch 3/3
1641/1641 [==============================] - 7s 4ms/step - loss: 0.7861 - accuracy: 0.6003 - val_loss: 0.8971 - val_accuracy: 0.4697


In [56]:
train_iteration(lr=2e-3, epochs=5)

=== Training with lr=0.002 for 5 epochs ===
Epoch 1/5
1641/1641 [==============================] - 12s 6ms/step - loss: 0.5465 - accuracy: 0.6838 - val_loss: 0.4457 - val_accuracy: 0.7209
Epoch 2/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.2529 - accuracy: 0.8989 - val_loss: 0.1400 - val_accuracy: 0.9593
Epoch 3/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.1121 - accuracy: 0.9668 - val_loss: 0.0919 - val_accuracy: 0.9741
Epoch 4/5
1641/1641 [==============================] - 7s 5ms/step - loss: 0.0929 - accuracy: 0.9724 - val_loss: 0.0772 - val_accuracy: 0.9777
Epoch 5/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0767 - accuracy: 0.9775 - val_loss: 0.0645 - val_accuracy: 0.9824


In [57]:
train_iteration(lr=1e-3, epochs=5)

=== Training with lr=0.001 for 5 epochs ===
Epoch 1/5
1641/1641 [==============================] - 13s 6ms/step - loss: 0.0639 - accuracy: 0.9824 - val_loss: 0.0602 - val_accuracy: 0.9831
Epoch 2/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0587 - accuracy: 0.9836 - val_loss: 0.0610 - val_accuracy: 0.9814
Epoch 3/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0559 - accuracy: 0.9846 - val_loss: 0.0550 - val_accuracy: 0.9845
Epoch 4/5
1641/1641 [==============================] - 6s 4ms/step - loss: 0.0539 - accuracy: 0.9850 - val_loss: 0.0512 - val_accuracy: 0.9855
Epoch 5/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0521 - accuracy: 0.9855 - val_loss: 0.0747 - val_accuracy: 0.9766


In [58]:
train_iteration(lr=2e-4, epochs=5)

=== Training with lr=0.0002 for 5 epochs ===
Epoch 1/5
1641/1641 [==============================] - 12s 6ms/step - loss: 0.0473 - accuracy: 0.9868 - val_loss: 0.0496 - val_accuracy: 0.9859
Epoch 2/5
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0460 - accuracy: 0.9871 - val_loss: 0.0492 - val_accuracy: 0.9864
Epoch 3/5
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0455 - accuracy: 0.9872 - val_loss: 0.0487 - val_accuracy: 0.9861
Epoch 4/5
1641/1641 [==============================] - 6s 4ms/step - loss: 0.0452 - accuracy: 0.9875 - val_loss: 0.0489 - val_accuracy: 0.9863
Epoch 5/5
1641/1641 [==============================] - 6s 4ms/step - loss: 0.0446 - accuracy: 0.9875 - val_loss: 0.0479 - val_accuracy: 0.9868


In [59]:
train_iteration(lr=2e-5, epochs=30)

=== Training with lr=2e-05 for 30 epochs ===
Epoch 1/30
1641/1641 [==============================] - 12s 6ms/step - loss: 0.0437 - accuracy: 0.9878 - val_loss: 0.0479 - val_accuracy: 0.9866
Epoch 2/30
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0435 - accuracy: 0.9878 - val_loss: 0.0480 - val_accuracy: 0.9865
Epoch 3/30
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0434 - accuracy: 0.9878 - val_loss: 0.0479 - val_accuracy: 0.9866
Epoch 4/30
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0434 - accuracy: 0.9878 - val_loss: 0.0478 - val_accuracy: 0.9867
Epoch 5/30
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0433 - accuracy: 0.9878 - val_loss: 0.0478 - val_accuracy: 0.9866
Epoch 6/30
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0433 - accuracy: 0.9878 - val_loss: 0.0478 - val_accuracy: 0.9867
Epoch 7/30
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0432 - accur

In [60]:
train_iteration(lr=1e-5, epochs=10)

=== Training with lr=1e-05 for 10 epochs ===
Epoch 1/10
1641/1641 [==============================] - 12s 6ms/step - loss: 0.0423 - accuracy: 0.9881 - val_loss: 0.0470 - val_accuracy: 0.9869
Epoch 2/10
1641/1641 [==============================] - 6s 4ms/step - loss: 0.0423 - accuracy: 0.9881 - val_loss: 0.0470 - val_accuracy: 0.9869
Epoch 3/10
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0422 - accuracy: 0.9881 - val_loss: 0.0471 - val_accuracy: 0.9869
Epoch 4/10
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0422 - accuracy: 0.9880 - val_loss: 0.0469 - val_accuracy: 0.9868
Epoch 5/10
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0422 - accuracy: 0.9881 - val_loss: 0.0470 - val_accuracy: 0.9869
Epoch 6/10
1641/1641 [==============================] - 7s 4ms/step - loss: 0.0422 - accuracy: 0.9881 - val_loss: 0.0471 - val_accuracy: 0.9869
Epoch 7/10
1641/1641 [==============================] - 8s 5ms/step - loss: 0.0422 - accur

In [61]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")

task name: axion2
INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v1.0/axion2_cnn/assets


INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v1.0/axion2_cnn/assets
